# Prospect demo: Visual inspection (VI) of DESI spectra within a notebook  

Works at NERSC, using DESI master kernel (includes prospect and bokeh). Updated July 2021.

- To create standalone VI pages (html files), run the script `prospect_pages` in command line (examples are in prospect/bin/examples_prospect_pages.sh)
- To inspect a list of TARGETIDs within a notebook, see the companion notebook `Prospect_targetid.ipynb`

In [ ]:
import os, sys
import numpy as np
from astropy.table import Table
import desispec

# If not using the desiconda version of prospect: EDIT THIS to your path
sys.path.insert(0, os.path.join(os.environ['HOME'], 'Documents', 'Code', 'git', 'desihub', 'prospect', 'py'))
from prospect import viewer, utilities

#### Prepare a set of spectra (`desispec.Spectra` objects)
Use filtering criteria at will.
<br>Prepare associated `astropy.table.Table` objects from zbest.fits and redrock.h5 files.

In [ ]:
# DESI-SV1 data: tile 80610 (ELG targets) in COSMOS, let's use fuji/EDR data release
specprod = os.environ['SPECPROD'] = 'fuji'
datadir = os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, 'tiles', 'cumulative', '80610', '20210208')
allcoadds = desispec.io.read_spectra(os.path.join(datadir, 'coadd-7-80610-thru20210208.fits'))
allzbest = Table.read(os.path.join(datadir, 'redrock-7-80610-thru20210208.fits'), hdu='REDSHIFTS')
print(allcoadds.num_spectra(),"spectra in file")

In [ ]:
# Select a subsample to be 'VIed':
zbest_select = allzbest[(allzbest["Z"]>1.3)&(allzbest["Z"]<1.5)&(allzbest["DELTACHI2"]>100)]
targets = zbest_select["TARGETID"]
coadds_select = allcoadds.select(targets=targets) # this is a desispec.spectra function
print(coadds_select.num_spectra(),"spectra selected")

In [ ]:
# Get full Redrock outputs (to get Nth best fits), matched to the former selection
redrockfile = os.path.join(datadir, 'rrdetails-7-80610-thru20210208.h5')
redrock_select = utilities.match_rrdetails_to_spectra(redrockfile, coadds_select)
# check 'redrock' and 'zbest' files are consistent:
print(np.array_equal(redrock_select['CHI2'][:,0], zbest_select['CHI2']))

#### VI tool

Tip: DO NOT load too many spectra into the viewer as it might overload your web browser (standard VI sets are made of 50 spectra).

Many options are available in `viewer.plotspectra`, here are some probably most useful:
- notebook=True is mandatory (otherwise will write an html file to disk at NERSC)
- mask_type (eg. 'SV2_SCND_TARGET', 'DESI_TARGET' is default) is important to display correctly the targeting masks for each spectrum.
- top_metadata: choose which metadata from spectra.fibermaps to highlight on the top panel

In [ ]:
# Run this cell to have the VI tool !
viewer.plotspectra(coadds_select, zcatalog=zbest_select, redrock_cat=redrock_select,
                   notebook=True, title='Prospect demo: ELG tile in SV1-COSMOS', 
                   mask_type='SV1_DESI_TARGET')

In [ ]:
# Full help on this function:
help(viewer.plotspectra)